In [1]:
import gurobipy as gp
import numpy as np

In [2]:
# Question 2

In [3]:
facMod=gp.Model()
facModX = facMod.addMVar(4, vtype = 'B')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-21


In [4]:
capital = [6,3,5,2]
profit = [9,5,6,4]

In [5]:
facMod.setObjective(gp.quicksum(profit[i]*facModX[i] for i in range(4)),sense=gp.GRB.MAXIMIZE)

In [6]:
facMod.addConstr(gp.quicksum(capital[i]*facModX[i] for i in range(4)) <= 11) #capital capacity
#facMod.addConstr(facModX[0] + facModX[2] <= 1) #up to one facility in Austin
#facMod.addConstr(facModX[1] + facModX[3] <= 1) #up to one facility in Dallas
facMod.addConstr(facModX[0] + facModX[1] >= 1) #at least one factory 
facMod.addConstr(facModX[2] + facModX[3] <= 1) #at most one warehouse

<MConstr () *awaiting model update*>

In [7]:
facMod.Params.OutputFlag = 0  #tell gurobi to shut up!!
facMod.optimize() # solve the model

In [8]:
facMod.ObjVal

18.0

In [9]:
print(facModX.x)

[1. 1. 0. 1.]


In [10]:
# Question 3

In [11]:
hubMod=gp.Model()
hubModX = hubMod.addMVar(12, vtype = 'B')

In [12]:
matrix = [[1,0,1,0,1,0,1,1,1,0,0,0],
          [0,1,0,0,0,0,0,1,1,0,0,0],
          [1,0,1,0,0,0,1,1,1,0,0,0],
          [0,0,0,1,0,0,0,0,0,1,0,0],
          [1,0,0,0,1,0,1,0,0,0,0,0],
          [0,0,0,0,0,1,0,0,0,1,1,0],
          [1,0,1,0,1,0,1,0,0,0,0,0],
          [1,1,1,0,0,0,0,1,1,0,0,0],
          [1,1,1,0,0,0,0,1,1,0,0,0],
          [0,0,0,1,0,1,0,0,0,1,1,1],
          [0,0,0,0,0,1,0,0,0,1,1,1],
          [0,0,0,0,0,0,0,0,0,1,1,1]]

In [13]:
hubMod.setObjective(gp.quicksum(hubModX[i] for i in range(12)),sense=gp.GRB.MINIMIZE)

In [14]:
hubMod.addConstrs((gp.quicksum(hubModX[i]*matrix[i][j] for i in range(12)) >= 1) for j in range(12))

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


{0: <MConstr ()>,
 1: <MConstr () *awaiting model update*>,
 2: <MConstr () *awaiting model update*>,
 3: <MConstr () *awaiting model update*>,
 4: <MConstr () *awaiting model update*>,
 5: <MConstr () *awaiting model update*>,
 6: <MConstr () *awaiting model update*>,
 7: <MConstr () *awaiting model update*>,
 8: <MConstr () *awaiting model update*>,
 9: <MConstr () *awaiting model update*>,
 10: <MConstr () *awaiting model update*>,
 11: <MConstr () *awaiting model update*>}

In [15]:
hubMod.Params.OutputFlag = 0  #tell gurobi to shut up!!
hubMod.optimize() # solve the model

In [16]:
hubMod.ObjVal

3.0

In [17]:
print(hubModX.x) #ATL, PIT, and SLC

[1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]


In [18]:
# Question 4

In [19]:
paperMod=gp.Model()
paperModX = paperMod.addMVar(18, vtype = 'I')

In [21]:
import itertools

# Define the prices of your three products
product_prices = [25, 37, 54]

# Define your budget
budget = 120

# Initialize an empty matrix to store combinations
combinations_matrix = []

# Loop through all possible quantities of each product
for quantity_1 in range(budget // product_prices[0] + 1):
    for quantity_2 in range(budget // product_prices[1] + 1):
        for quantity_3 in range(budget // product_prices[2] + 1):
            # Calculate the total cost for the current combination
            total_cost = (
                quantity_1 * product_prices[0] +
                quantity_2 * product_prices[1] +
                quantity_3 * product_prices[2]
            )
            
            # Check if the total cost is within the budget
            if total_cost <= budget:
                # Add the current combination to the matrix
                combinations_matrix.append([quantity_1, quantity_2, quantity_3])

# Print the matrix of combinations
for combination in combinations_matrix:
    print(combination)
    
wasteage = [budget - (combination[0]*product_prices[0] + combination[1]*product_prices[1] + combination[2]*product_prices[2])
            for combination in combinations_matrix]
wasteage

[0, 0, 0]
[0, 0, 1]
[0, 0, 2]
[0, 1, 0]
[0, 1, 1]
[0, 2, 0]
[0, 3, 0]
[1, 0, 0]
[1, 0, 1]
[1, 1, 0]
[1, 1, 1]
[1, 2, 0]
[2, 0, 0]
[2, 0, 1]
[2, 1, 0]
[3, 0, 0]
[3, 1, 0]
[4, 0, 0]


[120, 66, 12, 83, 29, 46, 9, 95, 41, 58, 4, 21, 70, 16, 33, 45, 8, 20]

In [24]:
width = [25,37,54]
quantity = [233,148,106]

opts = [[4,0,0],
        [3,1,0],
        [2,0,1],
        [1,2,0],
        [1,1,1],
        [0,3,0],
        [0,1,1],
        [0,0,2]]

In [22]:
paperMod.setObjective(gp.quicksum(paperModX[i]*wasteage[i] for i in range(18)),sense=gp.GRB.MINIMIZE)

In [25]:
paperMod.addConstrs((gp.quicksum(paperModX[i]*combinations_matrix[i][j] for i in range(18)) >= quantity[j]) for j in range(3))

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


{0: <MConstr ()>,
 1: <MConstr () *awaiting model update*>,
 2: <MConstr () *awaiting model update*>}

In [26]:
paperMod.Params.OutputFlag = 0  #tell gurobi to shut up!!
paperMod.optimize() # solve the model

In [27]:
paperMod.ObjVal

764.0

In [28]:
print(paperModX.x)

[ -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0. 107.  -0.  -0.  -0.
  -0.  -0.  42.  -0.]


In [ ]:
# Question 5

In [29]:
dosMod=gp.Model()
dosModX = dosMod.addMVar(7, vtype = 'I')

In [30]:
cost = [330,300,330,360,360,360,360]
required = [5,13,12,10,14,8,6]

In [31]:
dosMod.setObjective(gp.quicksum(cost[i]*dosModX[i] for i in range(7)),sense=gp.GRB.MINIMIZE)

In [32]:
dosMod.addConstrs((sum(dosModX[:]) - sum(dosModX[i+1:i+3]) >= required[i]) for i in range(5))
dosMod.addConstr(sum(dosModX[1:6]) >= required[5])
dosMod.addConstr(sum(dosModX[2:]) >= required[6])

<MConstr () *awaiting model update*>

In [33]:
dosMod.Params.OutputFlag = 0  #tell gurobi to shut up!!
dosMod.optimize() # solve the model

In [34]:
dosMod.ObjVal

4830.0

In [35]:
print(dosModX.x)

[ 1.  8.  2. -0.  3.  1. -0.]
